# EEP153 Project1 Team Ma Yinchu

[description of topic and goals]

In [272]:
## Imports used so far 
!pip install wbdata
import wbdata
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objects as go
import wbdata

In [273]:
## Imports for plots
# !pip install cufflinks
# import cufflinks as cf
# cf.go_offline()

## Project Deliverables 

### [#A] Population Statistics
A `python` function named population
that will deliver information to
answer the queries of the following
form:
> In [year] how many
[people/males/females] aged [low] to
[high] were living in [the
world/region/country]?

In [274]:
def population(year, sex, age_range, place):
    """ Takes 
    int `year`
    str `sex`
    tuple `age_range`
    str `wbdata` three-letter code `place`
    """
    
    # construct a list of age-ranges
    age_ranges = []
    for i in range(0,80,5):
        if i >= age_range[0] and i + 4 <= age_range[1]:
            age_ranges.append(f"{i:02d}"+f"{i+4:02d}")
        if (i <= age_range[0] <= i + 4) or (i <= age_range[1] <= i + 4):
            age_ranges.append(f"{i:02d}"+f"{i+4:02d}")
    if age_range[1] >= 80:
        age_ranges.append("80UP")
        
        
    # construct a dictionary of indicators, with labels, that we want to grab
    male = {"SP.POP."+age_range+".MA":"Males "+age_range for age_range in age_ranges}
    female = {"SP.POP."+age_range+".FE":"Females "+age_range for age_range in age_ranges}
    combined = {**male, **female}
    
    if sex == 'Male':
        variables = male
    elif sex == 'Female':
        variables = female
    elif sex == 'People':
        variables = combined
    
    # construct a dataframe, indexed by date, with population counts for each indicator, filtered by `place`
    df = wbdata.get_dataframe(variables, country="WLD")
    
    # sum all the desired population counts
    population_sum = sum(df.filter(items = [str(year)], axis=0).iloc[0])
    
    print(f"In {year} there were {population_sum} {sex}s aged {age_range[0]} to {age_range[1]} that were living in {place}.")
    
    return population_sum

Ex. In 2020, how many females aged 16 to 49 were living in Africa?

In [275]:
answer = population(2020, 'Female', (16, 49), 'AFE')

In 2020 there were 1920636605.0 Females aged 16 to 49 that were living in AFE.


### [#A] Population DataFrames

A function that returns a `pandas` DataFrame
indexed by Region or Country and Year, with
columns giving counts of people in different
age-sex groups.

In [276]:
def population_df(region):
    """Takes in 3 letter country code and returns a pandas DataFrame indexed by region and date, 
    with columns giving counts of people in different age-sex groups."""
    
    # Get all the indicator labels by age-sex bin and store in variable_labels dict
    variable_labels = {}
    for i in range(0, 80, 5):
        m_age = {"SP.POP." + f"{i:02d}" + f"{i+4:02d}" + ".MA": "M:" + f"{i:02d}" + "-" + f"{i+4:02d}"}
        f_age = {"SP.POP." + f"{i:02d}" + f"{i+4:02d}" + ".FE": "F:" + f"{i:02d}" + "-" + f"{i+4:02d}"}
        variable_labels.update(m_age)
        variable_labels.update(f_age)
    variable_labels.update({"SP.POP." + "80UP" + ".MA": "M:" + "80-UP"})
    variable_labels.update({"SP.POP." + "80UP" + ".FE": "F:" + "80-UP"})
    
    # Get population data for region input
    df = wbdata.get_dataframe(variable_labels, country = region)
    
    # Format DF and indices
    df.reset_index(inplace = True)
    df["Region"] = region
    df["date"] = df["date"].astype(int)
    df.set_index(["Region", "date"], inplace = True)
    return df

In [277]:
## Ex. Population dataframe for Africa
pop_df_afe = population_df("AFE")
pop_df_afe

M:00-04     F:00-04     M:05-09     F:05-09     M:10-14  \
Region date                                                               
AFE    2021  54474934.0  53234090.0  48590005.0  47750830.0  43631748.0   
       2020  53461851.0  52265991.0  47709312.0  46911818.0  42575598.0   
       2019  52445829.0  51296626.0  46830488.0  46072800.0  41476275.0   
       2018  51462324.0  50354295.0  45959995.0  45238285.0  40339961.0   
       2017  50527950.0  49459497.0  45048010.0  44365105.0  39194652.0   
...                 ...         ...         ...         ...         ...   
       1964  13473056.0  13316059.0  10457881.0  10371024.0   8700624.0   
       1963  13062183.0  12915169.0  10138780.0  10058500.0   8455835.0   
       1962  12661185.0  12522816.0   9837633.0   9765428.0   8228078.0   
       1961  12271309.0  12140807.0   9547953.0   9486214.0   8020790.0   
       1960  11903046.0  11781303.0   9269956.0   9224178.0   7827606.0   

                F:10-14     M:15-19     F:15-19     M:20-24     F:20-24  ...  \
Region date                                                              ...   
AFE    2021  43088865.0  37645093.0  37392619.0  32077598.0  32150758.0  ...   
       2020  42069398.0  36563052.0  36335804.0  31250943.0  31355868.0  ...   
       2019  40999618.0  35526881.0  35311876.0  30439607.0  30578412.0  ...   
       2018  39886015.0  34553287.0  34334491.0  29672727.0  29841888.0  ...   
       2017  38773390.0  33610378.0  33410378.0  28956812.0  29177045.0  ...   
...                 ...         ...         ...         ...         ...  ...   
       1964   8635913.0   7371925.0   7348233.0   6056915.0   6192034.0  ...   
       1963   8388190.0   7145770.0   7137094.0   5879098.0   6019568.0  ...   
       1962   8159040.0   6916635.0   6922855.0   5725665.0   5869666.0  ...   
       1961   7950546.0   6684830.0   6705676.0   5586353.0   5730412.0  ...   
       1960   7760047.0   6464521.0   6498502.0   5452406.0   5594132.0  ...   

               M:60-64    F:60-64    M:65-69    F:65-69    M:70-74    F:70-74  \
Region date                                                                     
AFE    2021  5530561.0  6828373.0  4019492.0  5147052.0  2724146.0  3599557.0   
       2020  5359947.0  6613487.0  3937852.0  5007885.0  2665421.0  3491072.0   
       2019  5191655.0  6396462.0  3835788.0  4848496.0  2579929.0  3362927.0   
       2018  5031828.0  6185332.0  3722030.0  4677211.0  2477747.0  3224304.0   
       2017  4892796.0  5991427.0  3609954.0  4509749.0  2373716.0  3086225.0   
...                ...        ...        ...        ...        ...        ...   
       1964  1257129.0  1387855.0   850939.0  1007921.0   541871.0   672056.0   
       1963  1213489.0  1349018.0   826719.0   983494.0   530004.0   655650.0   
       1962  1169274.0  1311157.0   805802.0   960546.0   519280.0   640343.0   
       1961  1126932.0  1275145.0   786369.0   938330.0   509464.0   626362.0   
       1960  1089990.0  1242066.0   767560.0   916159.0   500885.0   614569.0   

               M:75-79    F:75-79    M:80-UP    F:80-UP  
Region date                                              
AFE    2021  1559724.0  2207210.0  1085329.0  1909063.0  
       2020  1511511.0  2131769.0  1077480.0  1872417.0  
       2019  1452252.0  2048886.0  1050938.0  1814665.0  
       2018  1392035.0  1967877.0  1014714.0  1750682.0  
       2017  1338098.0  1894203.0   980011.0  1688213.0  
...                ...        ...        ...        ...  
       1964   302911.0   388984.0   196377.0   268313.0  
       1963   298421.0   382154.0   193690.0   263206.0  
       1962   294329.0   375946.0   191294.0   258394.0  
       1961   290320.0   369833.0   188995.0   253612.0  
       1960   286341.0   363808.0   186622.0   248982.0  

[62 rows x 34 columns]

### [#B] Population Pyramids

A `python` function that takes as
input a `pandas` DataFrame with
columns providing counts of people by
age-sex groups, and constructs a
“population pyramid” graph for
visualizing the data.

In [278]:
def population_pyramid(df, year):
    """ Takes
    `df` a dataframe indexed by region and year, with columns''
    `year` an int
    """
    
    py.init_notebook_mode(connected=True)
    
    # reset dataframe index to date/year
    df2 = df.reset_index(level='Region', drop=True)
    
    age_ranges = []
    for i in range(0,80,5):
        age_ranges.append(f"{i:02d}"+f"{i+4:02d}")

    age_ranges.append("80UP")


    layout = go.Layout(yaxis=go.layout.YAxis(range=[0, 90], title='Age'),
                   xaxis=go.layout.XAxis(
                       title='Number'),
                   barmode='overlay',
                   bargap=0.0)

    data = [go.Bar(y=[int(s[:2])+1 for s in age_ranges],
               x=df2.loc[year,:].filter(regex="M").values,
               orientation='h',
               name='Male',
               hoverinfo='x',
               marker=dict(color='powderblue')
               ),
        go.Bar(y=[int(s[:2])+1 for s in age_ranges],
               x=-df2.loc[year,:].filter(regex="F").values,
               orientation='h',
               name='Female',
               hoverinfo='x',
               marker=dict(color='mediumpurple')
               )]
    
    py.iplot(dict(data=data, layout=layout))
    
    return data


In [279]:
## Ex. Population pyramid for Africa in 2020
pop_pyramid_afe_data = population_pyramid(pop_df_afe, 2020)

### [#C] Animated Population Pyramids 

A `python` function that takes as
input a `pandas` DataFrame with
columns providing counts of people by
age-sex groups, with rows
corresponding to different years, and
constructs an animated “population
pyramid” graph for visualizing how the
population changes over time.

In [280]:
# construct list of age-ranges
age_ranges = []

for i in range(0,80,5):
    age_ranges.append(f"{i:02d}"+f"{i+4:02d}")

age_ranges.append("80UP")

In [281]:
# create a dictionary of indicators
male_variables = {"SP.POP."+age_range+".MA":"Males "+age_range for age_range in age_ranges}
female_variables = {"SP.POP."+age_range+".FE":"Females "+age_range for age_range in age_ranges}
variables = male_variables
variables.update(female_variables)

In [282]:
df = wbdata.get_dataframe(variables, country="AFE", cache = False)
df

,Males 0004,Males 0509,Males 1014,Males 1519,Males 2024,Males 2529,Males 3034,Males 3539,Males 4044,Males 4549,...,Females 3539,Females 4044,Females 4549,Females 5054,Females 5559,Females 6064,Females 6569,Females 7074,Females 7579,Females 80UP
date,,,,,,,,,,,,,,,,,,,,,
2021,54474934.0,48590005.0,43631748.0,37645093.0,32077598.0,27554198.0,23787250.0,19836395.0,15770587.0,12307798.0,...,20373836.0,16114329.0,12840580.0,10928926.0,8791191.0,6828373.0,5147052.0,3599557.0,2207210.0,1909063.0
2020,53461851.0,47709312.0,42575598.0,36563052.0,31250943.0,26824252.0,23186536.0,19161081.0,15076634.0,11954285.0,...,19607075.0,15395913.0,12524773.0,10625703.0,8505650.0,6613487.0,5007885.0,3491072.0,2131769.0,1872417.0
2019,52445829.0,46830488.0,41476275.0,35526881.0,30439607.0,26157013.0,22540445.0,18485600.0,14415738.0,11620415.0,...,18847517.0,14730960.0,12243730.0,10298215.0,8217134.0,6396462.0,4848496.0,3362927.0,2048886.0,1814665.0
2018,51462324.0,45959995.0,40339961.0,34553287.0,29672727.0,25565705.0,21859093.0,17803395.0,13828439.0,11294381.0,...,18101141.0,14153659.0,11978368.0,9960406.0,7931174.0,6185332.0,4677211.0,3224304.0,1967877.0,1750682.0
2017,50527950.0,45048010.0,39194652.0,33610378.0,28956812.0,25005637.0,21167835.0,17103010.0,13331062.0,10961721.0,...,17350716.0,13678514.0,11705198.0,9626161.0,7652897.0,5991427.0,4509749.0,3086225.0,1894203.0,1688213.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,13473056.0,10457881.0,8700624.0,7371925.0,6056915.0,5140702.0,4429617.0,3793506.0,3196747.0,2634712.0,...,3908192.0,3255916.0,2680389.0,2196325.0,1777026.0,1387855.0,1007921.0,672056.0,388984.0,268313.0
1963,13062183.0,10138780.0,8455835.0,7145770.0,5879098.0,5026028.0,4340514.0,3713247.0,3127933.0,2575964.0,...,3808828.0,3174922.0,2612835.0,2143589.0,1734161.0,1349018.0,983494.0,655650.0,382154.0,263206.0
1962,12661185.0,9837633.0,8228078.0,6916635.0,5725665.0,4918510.0,4253645.0,3635546.0,3060163.0,2520717.0,...,3714069.0,3095416.0,2550083.0,2092686.0,1693058.0,1311157.0,960546.0,640343.0,375946.0,258394.0


In [283]:
# create initial pyramid
year = 1960
data = [go.Bar(x = df.loc[str(year),:].filter(regex="Male").values,
           y = [int(s[:2])+1 for s in age_ranges],
           orientation='h',
           name='Men',
           marker=dict(color='green'),
           hoverinfo='skip'
           ),
    go.Bar(x = -df.loc[str(year),:].filter(regex="Female").values,
           y=[int(s[:2])+1 for s in age_ranges],
           orientation='h',
           name='Women',
           marker=dict(color='pink'),
           hoverinfo='skip',
           )
    ]


In [284]:
# create a layout with our title 
layout = go.Layout(barmode='overlay',
               yaxis=go.layout.YAxis(range=[0, 90], title='Age'),
               xaxis=go.layout.XAxis(title='Number'))


In [285]:
# combine the graph_objects into a figure
fig = go.Figure(data=data)

In [286]:
# to see what we have so far
fig.show()

In [287]:
# create a list of frames
frames = []

# create a frame for every line y
for year in df.index.sort_values():
    bins = [go.Bar(x = df.loc[str(year),:].filter(regex="Male").values,
               y = [int(s[:2])+1 for s in age_ranges],
               orientation='h',
               name='Men',
               marker=dict(color='green'),
               hoverinfo='skip'
               ),
        go.Bar(x = -df.loc[str(year),:].filter(regex="Female").values,
               y=[int(s[:2])+1 for s in age_ranges],
               orientation='h',
               name='Women',
               marker=dict(color='pink'),
               hoverinfo='skip',
               )
        ]
    
    # create the button
    button = {
        "type": "buttons",
        "buttons": [
            {
                "label": "Play",
                "method": "animate",
                "args": [None, {"frame": {"duration": 20}}],
            }
        ],
    }
    
    # add the button to the layout and update the 
    # title to show the gradient descent step
    layout = go.Layout(updatemenus=[button], 
                       title_text=f"Population Pyramid for {year}")
    
    # create a frame object
    frame = go.Frame(
        data= bins, 
        layout=go.Layout(title_text=f"Population Pyramid for {year}")
    )
    
    # add the frame object to the frames list
    frames.append(frame)

In [288]:
# combine the graph_objects into a figure
fig = go.Figure(data=data,
                frames=frames,
                layout = layout)
                                 
# show our animation!
fig.show()